<a href="https://colab.research.google.com/github/johnantonn/deep-learning-practice/blob/main/tensor_ops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor Operations
This notebook contains the definition of tensors along with a number of tensor operations in numpy.

All transformations learned by neural networkscan be reduced to a handful of tensor operations that are applied to tensors of numeric data:
- Element-wise operations
- Broadcasting
- Tensor product
- Tensor reshaping

Finally, there is a block of code providing a geometric interpretation of tensor operations. 

In [16]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

## Tensors

A tensor is a container for numerical data.

Tensors are defined by three key attributes:
- Number of axes (rank) or `ndim`: the number of dimensions
- Shape: tuple of integers that describes how many dimensions there are along each axis.
- Data type (or `dtype`): the type of data contained in the tensor e.g. `float32`, `float64` etc.

In [14]:
# rank-0 tensor (scalar)
tensor_r0 = np.array(10)
print("Rank:", tensor_r0.ndim)
print("Shape:", tensor_r0.shape)
print("- - - - - - - - - - - -")

# rank-1 tensor
tensor_r1 = np.array([1, 2, 3, 4, 5])
print("Rank:", tensor_r1.ndim)
print("Shape:", tensor_r1.shape)
print("- - - - - - - - - - - -")


# rank-2 tensor
tensor_r2 = np.array([[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]])
print("Rank:", tensor_r2.ndim)
print("Shape:", tensor_r2.shape)
print("- - - - - - - - - - - -")

# rank-3 tensor
tensor_r3 = np.array([[[1,2,3,4,5], [1,2,3,4,5]]])
print("Rank:", tensor_r3.ndim)
print("Shape:", tensor_r3.shape)
print("- - - - - - - - - - - -")

 # MNIST dataset example
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Rank:", x_train.ndim)
print("Shape:", x_train.shape)

Rank: 0
Shape: ()
- - - - - - - - - - - -
Rank: 1
Shape: (5,)
- - - - - - - - - - - -
Rank: 2
Shape: (2, 5)
- - - - - - - - - - - -
Rank: 3
Shape: (1, 2, 5)
- - - - - - - - - - - -
Rank: 3
Shape: (60000, 28, 28)


## Element-wise operations

In this code block naive implementations of `relu` and `add` are provided and compared against the vectorized `numpy` implementations.

In [19]:
# naive ReLU
def naive_relu(x):
  assert len(x.shape) == 2
  x = x.copy()
  for i in range(x.shape[0]):
    for j in range(x.shape[1]):
      x[i, j] = max(x[i, j], 0)
  return x

# naive add
def naive_add(x, y):
  assert len(x.shape) == 2
  assert x.shape == y.shape
  x = x.copy()
  for i in range(x.shape[0]):
    for j in range(x.shape[1]):
      x[i, j] += y[i, j]
  return x

# toy data
x = np.random.random((20, 100))
y = np.random.random((20, 100))

# naive implementations
t0 = time.time()
for _ in range(1000):
  z = naive_add(x, y)
  z = naive_relu(z)
print("Naive took: {0:.2f} s".format(time.time() - t0))

# numpy vectorized implementations
t1 = time.time()
for _ in range(1000):
  z = x + y
  z = np.maximum(z, 0)
print("Vectorized took: {0:.2f} s".format(time.time() - t1))

Naive took: 2.29 s
Vectorized took: 0.01 s


## Broadcasting

Broadcasting facilitates numerical operations between tensors of different shapes.

In [31]:
# toy data
x = np.random.random((40, 10))
y = np.random.random((10,))

# broadcasting intuition
Y = np.expand_dims(y, axis=0) # expand the dimensions of y
Y = np.concatenate([y] * 40, axis=0) # repeat y 40 times along axis=0
print("Shape of x:", x.shape)
print("Shape of Y:", Y.shape)

# naive implementation of adding a matrix (rank-2 tensor) and a vector (rank-1 tensor)
def naive_add_matrix_and_vector(x, y):
  assert len(x.shape) == 2
  assert len(y.shape) == 1
  assert x.shape[1] == y.shape[0]
  x = x.copy()
  for i in range(x.shape[0]):
    for j in range(x.shape[1]):
      x[i, j] += y[j]
  return x

# example
print("Shape of (x+y):", naive_add_matrix_and_vector(x, y).shape)

Shape of x: (40, 10)
Shape of Y: (400,)
Shape of (x+y): (40, 10)


## Tensor product 

The *tensor product* or *dot product* (not to be confused with the element-wise product) is one of the most common and useful tensor operations. In numpy, a tensor product is done using the `np.dot` function.

In [40]:
# vector-vector dot product
def naive_vector_dot(x, y):
  assert len(x.shape) == 1
  assert len(y.shape) == 1
  assert x.shape[0] == y.shape[0]
  z = 0.
  for i in range(x.shape[0]):
    z += x[i] * y[i]
  return z

# toy data
x = np.random.random((10,))
y = np.random.random((10,))

# z is a scalar
z = naive_vector_dot(x, y)
print(z)
print("- - - - - - - - - ")

# matrix-vector dot product
def naive_matrix_vector_dot(x, y):
  z = np.zeros(x.shape[0])
  for i in range(x.shape[0]):
    z[i] = naive_vector_dot(x[i, :], y)
  return z

# toy data
x = np.random.random((5, 10))
y = np.random.random((10,))

# z is a vector
z = naive_matrix_vector_dot(x, y)
print(z)
print("- - - - - - - - - ")


# matrix-matrix dot product
def naive_matrix_dot(x, y):
  assert len(x.shape) == 2
  assert len(y.shape) == 2
  assert x.shape[1] == y.shape[0]
  z = np.zeros((x.shape[0], y.shape[1]))
  for i in range(x.shape[0]):
    for j in range(y.shape[1]):
      row_x = x[i, :]
      col_y = y[:, j]
      z[i, j] = naive_vector_dot(row_x, col_y)
  return z

# toy data
x = np.random.random((4, 10))
y = np.random.random((10, 2))

# z is a 4x2 matrix
z = naive_matrix_dot(x, y)
print(z)

2.167749553552792
- - - - - - - - - 
[4.68566983 4.21193009 2.80086741 3.01636514 1.84756883]
- - - - - - - - - 
[[1.94669792 1.36877593]
 [2.78408974 2.9120038 ]
 [2.71824168 2.65759662]
 [2.31653082 2.37440425]]


## Tensor reshaping

This type of operation is usually applied for preprocessing purposes, i.e. when there's a need to change to rearrange the dimensions/values of an existing tensor.

In [47]:
# toy data matrix of shape 3x2
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print("Shape of x:", x.shape)
print(x)
print("- - - - - ")

# reshape to 2x3
x = x.reshape((2, 3))
print("Shape of x after reshape:", x.shape)
print(x)
print("- - - - - ")

# reshape to 6x1
x = x.reshape((6, 1))
print("Shape of x after reshape:", x.shape)
print(x)
print("- - - - - ")

Shape of x: (3, 2)
[[0. 1.]
 [2. 3.]
 [4. 5.]]
- - - - - 
Shape of x after reshape: (2, 3)
[[0. 1. 2.]
 [3. 4. 5.]]
- - - - - 
Shape of x after reshape: (6, 1)
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]]
- - - - - 


## Geometric interpretation of tensor operations

This section provides an intuitive graphical interpretation of tensor operations, which essentially apply a geometric transformation to the input tensor.

Deep neural networks consist entirely of chains of tensor operations that are just simple geometric transformations of the input data, it follows that they can be interpreted as a very complex geometric transformation in a high-dimensional space, implemented via a series of simpler steps.